In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle, utils

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [3]:
d_items     = pd.read_parquet(icu+'d_items.parquet')
inputevents = pd.read_parquet(icu+'inputevents.parquet')
icustays    = pd.read_parquet(icu+'icustays.parquet')

In [4]:
d_antibiotics = d_items[(d_items['linksto']=='inputevents')&(d_items['category']=='Antibiotics')]
d_antibiotics = d_antibiotics[['itemid','label']]
d_antibiotics.reset_index(inplace=True)

# non-Antibiotics 제거
d_antibiotics = d_antibiotics[~d_antibiotics['itemid'].isin([
    225898,225877,225895,225868,225869,225885,225905,225838,
    225848,225844,225896,225837,225873,228003,225871,225903])]
d_antibiotics.dropna(subset='itemid',inplace=True)
d_antibiotics.reset_index(drop=True,inplace=True)

# I. Inject antibiotics at least one time.

In [7]:
cohort = inputevents[inputevents['itemid'].isin(d_antibiotics.itemid.unique())].stay_id.unique()
len(cohort)

44426

In [8]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
for i in labvalues.abbreviation.unique():
    globals()['resample_{}'.format(i)] = pd.read_parquet('./data/resample/resample_%s.parquet'%i)
    globals()['resample_{}'.format(i)] = globals()['resample_{}'.format(i)][globals()['resample_{}'.format(i)]['stay_id'].isin(cohort)]

In [14]:
for i in labvalues.abbreviation.unique():
    print("%s:"%i,globals()['resample_{}'.format(i)].valuenum.isna().sum())

Alb: 1798804
Alk_Phos: 1329297
AG: 10080
BUN: 9330
Ca: 79784
CK: 2635463
D_Bil: 3931379
Glu: 11616
HCT: 9549
INR: 181246
PH: 1020639
PHOS: 78583
Platelet: 10500
Cl: 8911
SCr: 9292
nan: 8910
Potassium: 9316
T_Bil: 1304215
WBC: 10619
Gl: 11616
Mg: 32389
Ca_ion: 1199155
HCO3: 9104
AST: 1297942
ALT: 1297842
PTT: 190399
baseexcess: 1029613
lactate: 676211


In [13]:
for i in labvalues.abbreviation.unique():
    globals()['resample_{}'.format(i)] = utils.resample_ffill(globals()['resample_{}'.format(i)])
    globals()['resample_{}'.format(i)] = utils.resample_bfill(globals()['resample_{}'.format(i)])

In [15]:
resample_AST[resample_AST['valuenum'].isna()]

,charttime,valuenum,subject_id,hadm_id,stay_id,itemid
0,2189-06-27 08:42:00,NaN,10000980,26913865,39765666,220587
1,2189-06-27 09:42:00,NaN,10000980,26913865,39765666,220587
2,2189-06-27 10:42:00,NaN,10000980,26913865,39765666,220587
3,2189-06-27 11:42:00,NaN,10000980,26913865,39765666,220587
4,2189-06-27 12:42:00,NaN,10000980,26913865,39765666,220587
...,...,...,...,...,...,...
19,2139-10-11 14:18:00,NaN,19999625,25304202,31070865,220587
20,2139-10-11 15:18:00,NaN,19999625,25304202,31070865,220587
21,2139-10-11 16:18:00,NaN,19999625,25304202,31070865,220587
22,2139-10-11 17:18:00,NaN,19999625,25304202,31070865,220587
